# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda II

Vamos continuar trabalhando com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [16]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import statsmodels.formula.api as smf
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from varname import nameof

In [3]:
df = pd.read_csv('previsao_de_renda.csv')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             15000 non-null  int64  
 1   data_ref               15000 non-null  object 
 2   id_cliente             15000 non-null  int64  
 3   sexo                   15000 non-null  object 
 4   posse_de_veiculo       15000 non-null  bool   
 5   posse_de_imovel        15000 non-null  bool   
 6   qtd_filhos             15000 non-null  int64  
 7   tipo_renda             15000 non-null  object 
 8   educacao               15000 non-null  object 
 9   estado_civil           15000 non-null  object 
 10  tipo_residencia        15000 non-null  object 
 11  idade                  15000 non-null  int64  
 12  tempo_emprego          12427 non-null  float64
 13  qt_pessoas_residencia  15000 non-null  float64
 14  renda                  15000 non-null  float64
dtypes:

1. Separe a base em treinamento e teste (25% para teste, 75% para treinamento).
2. Rode uma regularização *ridge* com alpha = [0, 0.001, 0.005, 0.01, 0.05, 0.1] e avalie o $R^2$ na base de testes. Qual o melhor modelo?
3. Faça o mesmo que no passo 2, com uma regressão *LASSO*. Qual método chega a um melhor resultado?
4. Rode um modelo *stepwise*. Avalie o $R^2$ na vase de testes. Qual o melhor resultado?
5. Compare os parâmetros e avalie eventuais diferenças. Qual modelo você acha o melhor de todos?
6. Partindo dos modelos que você ajustou, tente melhorar o $R^2$ na base de testes. Use a criatividade, veja se consegue inserir alguma transformação ou combinação de variáveis.
7. Ajuste uma árvore de regressão e veja se consegue um $R^2$ melhor com ela.

In [5]:
df_encoded = df.drop(columns=['Unnamed: 0', 'data_ref', 'id_cliente'])
df_encoded.dropna(inplace=True)

## Com o dataframe ja limpo removendo colunas com dados faltantes (esses dados faltantes implicam em falhas nos modelos e metodos utilizados)

### Aqui eu separo a base de dados em treino e teste

In [6]:
df_treino, df_teste = train_test_split(df_encoded, test_size=0.25, random_state=4)

### No codigo abaixo eu gero o modelo e como pede o enunciado, gero um modelo para cada valor de alpha solicitado e com o L1_wt = 0 que representa o metodo *RIDGE*

In [13]:
modelo = "np.log(renda) ~ C(sexo, Treatment('F')) + C(posse_de_veiculo, Treatment(False)) + C(posse_de_imovel, Treatment(True)) + qtd_filhos + C(tipo_renda, Treatment('Assalariado')) + C(educacao, Treatment('Secundário')) + C(estado_civil, Treatment('Casado')) + C(tipo_residencia, Treatment('Casa')) + idade + tempo_emprego + qt_pessoas_residencia"
md = smf.ols(modelo, data = df_treino)
reg_alpha000 = md.fit_regularized(method = 'elastic_net', refit = True, L1_wt = 0, alpha = 0)

reg_alpha001 = md.fit_regularized(method = 'elastic_net', refit = True, L1_wt = 0, alpha = 0.001)

reg_alpha005 = md.fit_regularized(method = 'elastic_net', refit = True, L1_wt = 0, alpha = 0.005)

reg_alpha010 = md.fit_regularized(method = 'elastic_net', refit = True, L1_wt = 0, alpha = 0.01)

reg_alpha050 = md.fit_regularized(method = 'elastic_net', refit = True, L1_wt = 0, alpha = 0.05)

reg_alpha100 = md.fit_regularized(method = 'elastic_net', refit = True, L1_wt = 0, alpha = 0.1)

### Aqui eu avalio o R² de cada modelo usando um loop *for*

In [14]:
# quando o parametro L1_wt for igual a 1, utilizar esse bloco, se 0 utilizar o bloco seguinte a esse

y_true = df_teste
reg = [
    reg_alpha000,
    reg_alpha001,
    reg_alpha005,
    reg_alpha010,
    reg_alpha050,
    reg_alpha100
]
for y in reg:
    y_pred = y.predict(df_teste)
    y_pred_exp = np.exp(y_pred)
    print(f'R² do modelo nos dados de TREINO: {y.fit().rsquared_adj:.4f}')
    print(f"R² do modelo nos dados de TESTE: {r2_score(y_true['renda'], y_pred_exp):.4f}")
    print("-" * 60 )

AttributeError: 'RegularizedResults' object has no attribute 'fit'

In [45]:
reg = {
    'modelo_alpha_0.000': reg_alpha000,
    'modelo_alpha_0.001': reg_alpha001,
    'modelo_alpha_0.005': reg_alpha005,
    'modelo_alpha_0.01': reg_alpha010,
    'modelo_alpha_0.05': reg_alpha050,
    'modelo_alpha_0.1': reg_alpha100
}
for y in reg:
    y_pred_teste = reg[y].predict(df_teste)
    y_pred_treino = reg[y].predict(df_treino)
    y_pred_treino_exp = np.exp(y_pred_treino)
    y_pred_teste_exp = np.exp(y_pred_teste)
    print(f'R² do modelo nos dados de TREINO: ({y}) {r2_score(df_treino['renda'], y_pred_treino_exp):.4f}')
    print(f"R² do modelo nos dados de TESTE: ({y}) {r2_score(df_teste['renda'], y_pred_teste_exp):.4f}")
    print("-" * 60 )
    

R² do modelo nos dados de TREINO: (modelo_alpha_0.000) 0.2662
R² do modelo nos dados de TESTE: (modelo_alpha_0.000) 0.2795
------------------------------------------------------------
R² do modelo nos dados de TREINO: (modelo_alpha_0.001) 0.2674
R² do modelo nos dados de TESTE: (modelo_alpha_0.001) 0.2839
------------------------------------------------------------
R² do modelo nos dados de TREINO: (modelo_alpha_0.005) 0.2733
R² do modelo nos dados de TESTE: (modelo_alpha_0.005) 0.3050
------------------------------------------------------------
R² do modelo nos dados de TREINO: (modelo_alpha_0.01) 0.2731
R² do modelo nos dados de TESTE: (modelo_alpha_0.01) 0.3240
------------------------------------------------------------
R² do modelo nos dados de TREINO: (modelo_alpha_0.05) -0.1335
R² do modelo nos dados de TESTE: (modelo_alpha_0.05) -0.2968
------------------------------------------------------------
R² do modelo nos dados de TREINO: (modelo_alpha_0.1) -6.3154
R² do modelo nos dado

### Avaliando os modelos acima e se baseando no R² dos testes, o modelo com aplha = 0.01 teve o melhor resultado

## 3. Agora fazendo para um modelo LASSO

In [48]:
modelo = "np.log(renda) ~ C(sexo, Treatment('F')) + C(posse_de_veiculo, Treatment(False)) + C(posse_de_imovel, Treatment(True)) + qtd_filhos + C(tipo_renda, Treatment('Assalariado')) + C(educacao, Treatment('Secundário')) + C(estado_civil, Treatment('Casado')) + C(tipo_residencia, Treatment('Casa')) + idade + tempo_emprego + qt_pessoas_residencia"
md = smf.ols(modelo, data = df_treino)
reg_alpha000 = md.fit_regularized(method = 'elastic_net', refit = True, L1_wt = 1, alpha = 0)

reg_alpha001 = md.fit_regularized(method = 'elastic_net', refit = True, L1_wt = 1, alpha = 0.001)

reg_alpha005 = md.fit_regularized(method = 'elastic_net', refit = True, L1_wt = 1, alpha = 0.005)

reg_alpha010 = md.fit_regularized(method = 'elastic_net', refit = True, L1_wt = 1, alpha = 0.01)

reg_alpha050 = md.fit_regularized(method = 'elastic_net', refit = True, L1_wt = 1, alpha = 0.05)

reg_alpha100 = md.fit_regularized(method = 'elastic_net', refit = True, L1_wt = 1, alpha = 0.1)

In [49]:
y_true = df_teste
reg = [
    reg_alpha000,
    reg_alpha001,
    reg_alpha005,
    reg_alpha010,
    reg_alpha050,
    reg_alpha100
]
for y in reg:
    y_pred = y.predict(df_teste)
    y_pred_exp = np.exp(y_pred)
    print(f'R² do modelo nos dados de TREINO: {y.fit().rsquared_adj:.4f}')
    print(f"R² do modelo nos dados de TESTE: {r2_score(y_true['renda'], y_pred_exp):.4f}")
    print("-" * 60 )

AttributeError: 'OLSResults' object has no attribute 'fit'